# **Conjunto de dados usado para estimar o número de mulheres editoras e suas práticas de edição na Wikipédia espanhola**


## Informação dos atributos:

**gender:** 0 (desconhecido), 1 (masculino), 2 (feminino)  
**C_api:** gênero extraído da API WikiMedia, códigos como feminino/masculino/desconhecido  
**C_man:** sexo extraído da codificação do conteúdo, codificado como 1 (masculino) / 2 (feminino) / 3 (desconhecido)  
**E_NEds:** Índice I do estrato IJ (0,1,2,3)  
**E_Bpag:** Índice J do estrato IJ (0,1,2,3)  
**firstDay:** primeira edição na Wikipédia espanhola (AAAAMMDDHHMMSS)  
**lastDay:** última edição na Wikipédia espanhola (AAAAMMDDHHMMSS)  
**NEds:** número total de edições  
**NDays:** número de dias (últimoDia-primeiroDia+1)  
**NActDays:** número de dias com edições  
**NPages:** número de páginas diferentes editadas  
**NPcreated:** número de páginas criadas  
**pagesWomen:** número de edições em páginas relacionadas a mulheres  
**wikiprojWomen:** número de edições em WikiProjects relacionadas a mulheres  
**ns_user:** número de edições no usuário do namespace  
**ns_wikipedia:** número de edições no namespace wikipedia  
**ns_talk:** número de edições no namespace talk  
**ns_userTalk:** número de edições no namespace user talk  
**ns_content:** número de edições nas páginas de conteúdo  
**weightIJ:** corrigindo o peso para o estrato IJ  
**NIJ:** número de elementos no estrato IJ  

### **Medidas**
**GENDER:** Foi determinado pela combinação do gênero especificado no perfil do usuário (através da API MediaWiki) com o gênero extraído da codificação de contetúdo. (ex: no perfil do usuário possui expressões como "sou advogada")  
**CAPI:** Dados da API MediaWiki  
**CMAN:** Gênero calculado pela codificação de conteúdo manual descrito no artigo  

Se **CMAN** = **CAPI**, então classe = **CMAN**  
Se **CMAN** = *unknown* & **CAPI** != *unknown*, então classe = **CAPI**  
Se **CMAN** != *unknown* & **CAPI** = *unkonwn*, então classe = **CMAN**  
Caso contrário, gere um erro, pois ambos os procedimentos não concordam. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preparação dos dados

In [ ]:
import pandas as pd
pd.set_option('max_columns', None)
df = pd.read_csv('data.csv')

In [ ]:
df.head()

,gender,C_api,C_man,E_NEds,E_Bpag,firstDay,lastDay,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content,weightIJ,NIJ
0,1,male,1,2,2,20170527205915,20170721044501,543,56,43,204,4,0,0,91,28,6,76,324,0.915024,978
1,0,unknown,3,3,1,20110301072441,20170731213735,2764,2345,514,722,7,0,0,100,249,183,646,1526,0.661673,477
2,1,male,1,0,2,20060907204302,20140911191722,57,2927,25,25,0,0,0,3,0,1,3,49,0.800528,664
3,1,male,1,1,2,20121003144916,20121208180528,104,67,5,66,2,0,0,20,1,2,2,78,1.027717,841
4,0,unknown,3,1,1,20070311125035,20141106121057,184,2798,27,125,0,0,0,26,10,5,24,112,0.997535,994


## Selecionando atributos relevantes
Removendo as colunas **[E_NEds | E_Bpag | weightIJ | NIJ]** porque são informações da técnica de amostragem de dados descritos no artigo, portanto, não é necessário para treinar os modelos de predição de dados.

In [ ]:
df = df[['gender', 'C_api', 'C_man', 'firstDay', 'lastDay', 'NEds', 'NDays', 'NActDays', 'NPages', 'NPcreated', 'pagesWomen', 'wikiprojWomen', 'ns_user', 'ns_wikipedia', 'ns_talk', 'ns_userTalk', 'ns_content']]

Removendo os atributos C_api e C_man para termos somente uma classe de gênero no dataset. O atributo gender foi definido como a única classe porque ela foi determinada pela combinação do gênero especificado no perfil do usuário (através da API MediaWiki) com o gênero extraído da codificação de contetúdo. (ex: no perfil do usuário possui expressões como "sou advogada") 

In [ ]:
df = df[['gender', 'firstDay', 'lastDay', 'NEds', 'NDays', 'NActDays', 'NPages', 'NPcreated', 'pagesWomen', 'wikiprojWomen', 'ns_user', 'ns_wikipedia', 'ns_talk', 'ns_userTalk', 'ns_content']]

## Transformando dados

Como observado na nossa exploração de dados, os atributos de data são de suma importância para o treinamento do modelo, visto que existe uma diferença entre os períodos observados na classe Masculino e Feminino. Portanto, vamos converter as datas que estão no formato **AAAAMMDDhhmmss** para **AAMM**, e posteriormente, aplicando a normalização desses dados.

In [ ]:
def date_to_AAAAMMDD(date):
  return str(date)[:6]

In [ ]:
df['firstDay'] = df['firstDay'].apply(date_to_AAAAMMDD)
df['lastDay']  = df['lastDay'].apply(date_to_AAAAMMDD)

In [ ]:
df.head()

,gender,firstDay,lastDay,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content
0,1,201705,201707,543,56,43,204,4,0,0,91,28,6,76,324
1,0,201103,201707,2764,2345,514,722,7,0,0,100,249,183,646,1526
2,1,200609,201409,57,2927,25,25,0,0,0,3,0,1,3,49
3,1,201210,201212,104,67,5,66,2,0,0,20,1,2,2,78
4,0,200703,201411,184,2798,27,125,0,0,0,26,10,5,24,112


## Normalizando os dados
Visto que os dados não são normalmente distribuidos, vamos a aplicar o Standard Scaler para  realizar a normalização de dados.  
Essa normalização utiliza a média e o desvio padrão, que é mais útil quando existem muitos outliers na base de dados.

In [ ]:
df.describe()

,gender,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content
count,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000,4746.000000
mean,0.737042,2029.969448,2036.607880,183.162663,689.451960,43.479140,0.438896,0.439949,74.372946,74.368310,49.947745,96.081753,1521.886641
std,0.585355,7793.300833,1336.119914,374.034481,3355.302483,297.395507,5.327440,17.832244,246.407233,560.782479,215.554281,545.025818,6099.009235
min,0.000000,50.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,95.000000,835.250000,24.000000,29.000000,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,61.000000
50%,1.000000,218.000000,2035.500000,53.000000,68.000000,4.000000,0.000000,0.000000,14.000000,1.000000,4.000000,5.000000,151.000000
75%,1.000000,757.750000,3146.500000,154.000000,219.750000,14.000000,0.000000,0.000000,46.000000,8.000000,19.000000,22.000000,563.750000
max,2.000000,153193.000000,5349.000000,3843.000000,94142.000000,13394.000000,185.000000,949.000000,6041.000000,24392.000000,4788.000000,12350.000000,115547.000000


### Separando os atributos previsores

In [ ]:
previsores = ['firstDay', 'lastDay', 'NEds', 'NDays', 'NActDays', 'NPages', 'NPcreated', 'pagesWomen', 'wikiprojWomen', 'ns_user', 'ns_wikipedia', 'ns_talk', 'ns_userTalk', 'ns_content']
df_class = df[['gender']]
df       = df[previsores]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

O tipo de dado muda, não é mais um DF, e sim um Numpy Array

In [ ]:
min(df_scaled[0]), max(df_scaled[0]), type(df_scaled)

(-1.4825140715257943, 2.0221692216156013, numpy.ndarray)

Convertendo Numpy Array para Pandas Dataframe e concatenando a classe Gender

In [ ]:
df_scaled = pd.DataFrame(data=df_scaled, columns = df.columns)
df_scaled = pd.concat([df_scaled, df_class], axis=1)
df_scaled.describe()

,firstDay,lastDay,NEds,NDays,NActDays,NPages,NPcreated,pagesWomen,wikiprojWomen,ns_user,ns_wikipedia,ns_talk,ns_userTalk,ns_content,gender
count,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4.746000e+03,4746.000000
mean,5.595094e-16,-5.313152e-15,2.574379e-17,-1.375498e-17,2.579058e-17,1.919381e-17,7.565237e-17,2.721637e-16,-5.079165e-17,-4.982085e-17,-1.296137e-16,1.860957e-16,2.076112e-16,1.451524e-17,0.737042
std,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,1.000105e+00,0.585355
min,-2.255495e+00,-1.989587e+00,-2.540872e-01,-1.523682e+00,-4.870724e-01,-2.052049e-01,-1.462151e-01,-8.239270e-02,-2.467417e-02,-2.978024e-01,-1.326292e-01,-2.317421e-01,-1.763070e-01,-2.495564e-01,0.000000
25%,-8.248665e-01,-8.110295e-01,-2.483124e-01,-8.992340e-01,-4.255742e-01,-1.968590e-01,-1.428522e-01,-8.239270e-02,-2.467417e-02,-2.856262e-01,-1.326292e-01,-2.317421e-01,-1.744720e-01,-2.395538e-01,0.000000
50%,-2.332746e-01,3.332011e-01,-2.325280e-01,-8.292647e-04,-3.480331e-01,-1.852344e-01,-1.327636e-01,-8.239270e-02,-2.467417e-02,-2.450387e-01,-1.308458e-01,-2.131834e-01,-1.671322e-01,-2.247957e-01,1.000000
75%,8.816484e-01,9.053164e-01,-1.632625e-01,8.307705e-01,-7.797606e-02,-1.400027e-01,-9.913481e-02,-8.239270e-02,-2.467417e-02,-1.151587e-01,-1.183619e-01,-1.435880e-01,-1.359377e-01,-1.571136e-01,1.000000
max,2.033546e+00,9.224799e-01,1.939858e+01,2.479374e+00,9.785790e+00,2.785513e+01,4.489620e+01,3.464714e+01,5.319914e+01,2.421705e+01,4.336832e+01,2.198310e+01,2.248556e+01,1.869765e+01,2.000000


# Separando o dataset entre gênero conhecido e desconhecido

Vamos separar o dataset entre o conjunto de dados que possuem gêneros definidos (Masculino e Feminino) e o conjunto de dados que possuem gêneros desconhecidos. Essa separação se faz necessária para podermos utilizar a técnica de desbalaceamento do dataset posteriormente. Como também, o dataset com as instâncias de classe "Desconhecida" vai ser utilizada em produção após o treinamento do modelo.

In [ ]:
df_scaled_unknown = df_scaled[df_scaled['gender'] == 0]
df_scaled_known   = df_scaled[df_scaled['gender'] != 0]

df_scaled_unknown.shape, df_scaled_known.shape

((1601, 15), (3145, 15))

# Balanceando o dataset de treino

Visto que os dados estão desbalanceados, nós vamos aplicar a técnica de Oversampling para replicar as observações com menor quantidade para se equalizar em números de classificação.  

Vamos utilizar o método SMOTE que encontra os vizinhos próximos para as classes em minoria para cada amostra das classififcações. Em seguida, traça uma reta entre o ponto original e o vizinho para definir a localização da observação da observação genérica.

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X, y = df_scaled_known[previsores], df_scaled_known['gender']

X_resampled, y_resampled = SMOTE().fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = .25, random_state=1)

In [ ]:
y_resampled.value_counts()

1    2792
2    2792
Name: gender, dtype: int64

In [ ]:
print(f'{X_train.shape[0]} dados para treinar o modelo e {X_test.shape[0]} para testar o modelo treinado.')

4188 dados para treinar o modelo e 1396 para testar o modelo treinado.


## Exportando dataset para realizar a predição e as variáveis de treino e teste

In [ ]:
path = '/content/drive/MyDrive/Mineração de Dados/IF998 - Missões SMD/Projeto/'

In [ ]:
import pickle
with open(path+'train_test.pkl', 'wb') as f:
    pickle.dump([X_train, X_test, y_train, y_test], f) 

In [ ]:
df_scaled_unknown.to_csv(path+'data_predict.csv')